In [ ]:
!pip install numpy pandas scikit-learn matplotlib ipywidgets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
from ipywidgets import interact, interactive, FloatSlider, Dropdown, IntSlider

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
def poison_attack(poison_percent=0.3, target_class=1, source_class=0):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Poison the data
    source_indices = np.where(y_train == source_class)[0]
    num_poison = int(poison_percent * len(source_indices))
    poison_indices = np.random.choice(source_indices, num_poison, replace=False)

    y_train_poisoned = y_train.copy()
    y_train_poisoned[poison_indices] = target_class  # Mislabel

    # Train poisoned model
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train_poisoned)
    y_pred = model.predict(X_test)

    # Train clean model
    clean_model = LogisticRegression(max_iter=200)
    clean_model.fit(X_train, y_train)
    clean_pred = clean_model.predict(X_test)

    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Poisoned confusion matrix
    cm_poison = confusion_matrix(y_test, y_pred)
    ax1.imshow(cm_poison, cmap='Blues')
    ax1.set_title(f"Poisoned Model (Accuracy: {accuracy_score(y_test, y_pred):.2f})")
    ax1.set_xticks([0, 1, 2], iris.target_names)
    ax1.set_yticks([0, 1, 2], iris.target_names)

    # Clean confusion matrix
    cm_clean = confusion_matrix(y_test, clean_pred)
    ax2.imshow(cm_clean, cmap='Greens')
    ax2.set_title(f"Clean Model (Accuracy: {accuracy_score(y_test, clean_pred):.2f})")
    ax2.set_xticks([0, 1, 2], iris.target_names)
    ax2.set_yticks([0, 1, 2], iris.target_names)

    plt.show()

    # output
    print(f"🔥 You poisoned {num_poison} {iris.target_names[source_class]} samples as {iris.target_names[target_class]}!")
    if accuracy_score(y_test, y_pred) < 0.8:
        print("💀 Critical hit! The model is now deeply confused.")
    else:
        print("⚠️ The model still kinda works... try poisoning more!")

In [ ]:
interact(
    poison_attack,
    poison_percent=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.3, description="Poison %:"),
    target_class=Dropdown(options=[0, 1, 2], description="Target Class:",
                          value=1,
                          tooltips=['Setosa', 'Versicolor', 'Virginica']),
    source_class=Dropdown(options=[0, 1, 2], description="Source Class:",
                          value=0,
                          tooltips=['Setosa', 'Versicolor', 'Virginica'])
)

interactive(children=(FloatSlider(value=0.3, description='Poison %:', max=1.0), Dropdown(description='Target C…

<function __main__.poison_attack(poison_percent=0.3, target_class=1, source_class=0)>

In [ ]:
__manual = True

import ipywidgets as widgets
button = widgets.Button(description="Surprise Me! (Random Attack)")
output = widgets.Output()

def on_button_click(b):
    with output:
        poison_percent = np.random.uniform(0.1, 0.8)
        source = np.random.choice([0, 1, 2])
        target = np.random.choice([i for i in [0, 1, 2] if i != source])
        print(f"🎲 Random attack: Poison {poison_percent:.0%} of {iris.target_names[source]} as {iris.target_names[target]}!")
        poison_attack(poison_percent, target, source)

button.on_click(on_button_click)
display(button, output)

Button(description='Surprise Me! (Random Attack)', style=ButtonStyle())

Output()